In [1]:
import pandas
import numpy as np
import xarray
import matplotlib.pyplot as plt
import ulmo
from ulmo.utils import image_utils
from ulmo.plotting import plotting
import seaborn as sns
import os
import cartopy

/Users/rachelstumpf/ulmo/ulmo/utils/image_utils.py:8: UserWarning: healpy not installed.  Not all functions will work!
  warnings.warn("healpy not installed.  Not all functions will work!")


In [2]:
# read data set from Nautilus files: list of 98% clear locations / dates

df = pandas.read_parquet('/Users/rachelstumpf/Documents/VIIRS_all_98clear_std.parquet')
# I want to be able to grab the file from my Aqua-1 hard drive that has equivalent latitude and longitude, 
# or maybe by id number ??


In [40]:
# need to grab date, time, and location... since dates have multiple clear images, 
# there will be multiple copies of each date from L3S data set, just at different locations
# I don't need time, can reduce to just date

extract_lat = df['lat']
extract_lon = df['lon']
extract_datetime = df['datetime']
extract_date = extract_datetime.dt.date

In [41]:
# okay, now I want to be able to extract one row

extract_lat[0]

19.337208

In [3]:
# and I want to match this with its xarray from this date
# so first I should read in an xarray w a specific date, and what I should do is go through all of that xarray 
# date to see if it has matches in the VIIRS Table

# let's do the first date: 2012-02-01

# how do we make sure that the same spot wasn't clear at the same date at multiple times ?? 
# don't want this to be repeats within the cutouts

data_xr = xarray.open_dataset("/Volumes/Aqua-1/Hackathon/daily/l3s_fields/2012/032/20120201120000-STAR-L3S_GHRSST-SSTsubskin-LEO_Daily-ACSPO_V2.80-v02.0-fv01.0.nc")

In [4]:
# now I want to extract pieces from VIIRS that match this time

df['date'] = df['datetime'].dt.date # this makes new column w/o time

date_extract = data_xr['time'].dt.date # this extracts the date we want from L3S
date_extract = pandas.Series(date_extract.values)
match = df[df['date'].isin(date_extract)] # this selects VIIRS rows that match L3S date

In [74]:
# extract 02-01-2012 from VIIRS table, to see if we get the same number of rows as a check, THIS WORKS !

target_date = pandas.to_datetime('2012-02-01')

filtered_df = df[(df['datetime'].dt.date == target_date.date())]

#date_array = xarray.decode_cf(data_xr)['time'].dt.date
#data_xr['time'] = date_array

In [76]:
# the match data set has all of the specific locations from that date, now we need to go through the xarray
# and extract each location that corresponds to a row in the VIIRS table

data_xr['sea_surface_temperature']

<xarray.DataArray 'sea_surface_temperature' (time: 1, lat: 9000, lon: 18000)>
[162000000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 89.99 89.97 89.95 89.93 ... -89.95 -89.97 -89.99
  * lon      (lon) float32 -180.0 -180.0 -179.9 -179.9 ... 179.9 180.0 180.0
  * time     (time) datetime64[ns] 2012-02-01T12:00:00
Attributes:
    comment:                SST obtained by regression with buoy measurements...
    long_name:              sea surface sub-skin temperature
    source:                 NOAA
    units:                  kelvin
    standard_name:          sea_surface_subskin_temperature
    grid_mapping:           crs
    coverage_content_type:  physicalMeasurement
    valid_min:              -32767
    valid_max:              32767

In [38]:
# for each ROW in VIIRS table... take that and make it the center of the xarray cutout

# for some reason it's not selecting accurate lon lat? 

pixel_num = 64
variable = data_xr['sea_surface_temperature']

results = []

for index, row in match.iterrows():
    
    centered_lat = row['lat']
    centered_lon = row['lon']
    
    lats = variable['lat'].values
    lons = variable['lon'].values
    
    center_lat = np.abs(lats-centered_lat).argmin()
    center_lon = np.abs(lons-centered_lon).argmin()
    
    start_lat = center_lat - (pixel_num // 2)
    end_lat = start_lat + pixel_num
    start_lon = center_lon - (pixel_num // 2)
    end_lon = start_lon + pixel_num
    
    subarray = variable.isel(lon = slice(start_lon, end_lon), lat = slice(start_lat, end_lat))
    nan_percentage = np.isnan(subarray.values).sum()
    
    results.append({"lat": centered_lat, "lon": centered_lon, "NaN":nan_percentage})

    #results.append(subarray)
    
results_df = pandas.DataFrame(results)

# Print the results table
print(results_df)

           lat        lon  NaN
0    19.337208 -176.89246    0
1    20.116112 -176.30530    0
2    21.009575 -175.17024    0
3    20.876402 -175.80142    0
4    20.725744 -176.50098    0
..         ...        ...  ...
816  24.445341 -154.81297    0
817  24.347338 -155.73036    0
818  24.256254 -156.53986    0
819  24.149857 -157.44040    0
820  30.722930 -171.59128    0

[821 rows x 3 columns]


In [36]:
results_df[results_df['NaN'] != 0]
66/(64*64) # even the one that may have the most masked pixels still only has 1.6% masked, this is under the 2% benchmark, as it should be

0.01611328125

In [ ]:
# how to store cutouts / should I try to plot them to see if they overlap?

xarray_list = []

pixel_num = 64
variable = data_xr['sea_surface_temperature']

for index, row in match.iterrows():
    
    centered_lat = row['lat']
    centered_lon = row['lon']
    
    lats = variable['lat'].values
    lons = variable['lon'].values
    
    center_lat = np.abs(lats-centered_lat).argmin()
    center_lon = np.abs(lons-centered_lon).argmin()
    
    start_lat = center_lat - (pixel_num // 2)
    end_lat = start_lat + pixel_num
    start_lon = center_lon - (pixel_num // 2)
    end_lon = start_lon + pixel_num
    
    subarray = variable.isel(lon = slice(start_lon, end_lon), lat = slice(start_lat, end_lat))
    nan_percentage = np.isnan(subarray.values).sum()
    
    data_xarray = xarray.Dataset(
        {
            'sea_suface_temperature':(['lat','lon'], subarray.values),
        },
        coords = {'lat': lat_data, 'lon': lon_data}
    )
    xarray_list.append(data_xarray)

In [59]:
centered_lat = 19.337208
centered_lon = -176.89246
    
lats = variable['lat'].values
lons = variable['lon'].values
    
center_lat = np.abs(lats-centered_lat).argmin()
center_lon = np.abs(lons-centered_lon).argmin()

start_lat = center_lat - (pixel_num // 2)
end_lat = start_lat + pixel_num
start_lon = center_lon - (pixel_num // 2)
end_lon = start_lon + pixel_num
    
subarray = variable.isel(lon = slice(start_lon, end_lon), lat = slice(start_lat, end_lat))
subarray_values = subarray.values.transpose()

In [56]:
data_xarray = xarray.Dataset(
    {
        'sea_surface_temperature': (['lat', 'lon'], subarray_values),
    },
    coords={'lat': subarray['lat'], 'lon': subarray['lon']}
)

ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (['lat', 'lon'], array([299.53998, 299.53   , 299.47998, ..., 299.66998, 299.63   ,
       299.63   ], dtype=float32)) to Variable.

In [45]:
subarray['lat']

<xarray.DataArray 'lat' (lat: 64)>
array([19.97, 19.95, 19.93, 19.91, 19.89, 19.87, 19.85, 19.83, 19.81, 19.79,
       19.77, 19.75, 19.73, 19.71, 19.69, 19.67, 19.65, 19.63, 19.61, 19.59,
       19.57, 19.55, 19.53, 19.51, 19.49, 19.47, 19.45, 19.43, 19.41, 19.39,
       19.37, 19.35, 19.33, 19.31, 19.29, 19.27, 19.25, 19.23, 19.21, 19.19,
       19.17, 19.15, 19.13, 19.11, 19.09, 19.07, 19.05, 19.03, 19.01, 18.99,
       18.97, 18.95, 18.93, 18.91, 18.89, 18.87, 18.85, 18.83, 18.81, 18.79,
       18.77, 18.75, 18.73, 18.71], dtype=float32)
Coordinates:
  * lat      (lat) float32 19.97 19.95 19.93 19.91 ... 18.77 18.75 18.73 18.71
Attributes:
    long_name:              latitude
    comment:                Latitudes for locating data
    units:                  degrees_north
    axis:                   Y
    valid_min:              -90.0
    valid_max:              90.0
    standard_name:          latitude
    coverage_content_type:  coordinate

In [ ]:

# Plot each subarray on a world map using Cartopy
for subarray_xr in results:
    # Create a figure and axis using Cartopy
    fig, ax = plt.subplots(subplot_kw={'projection': cartopy.crs.PlateCarree()})
    
    # Plot the subarray on the world map
    subarray_xr['sea_surface_temperature'].plot(ax=ax, transform=cartopy.crs.PlateCarree(), cmap='coolwarm')
    
    # Add map features
    ax.coastlines()
    ax.gridlines()
    
    # Show the plot
    plt.show()